In [16]:
import pickle
from statistics import mean
import numpy as np
import sys
import glob
import math
from scipy.special import comb

In [2]:
file_name_train_250 = "../Human Experiments/Data/RL_trajectories_1000_250.pickle"
file_obj_train_250 = open(file_name_train_250, 'rb')
trajectories_train_250 = pickle.load(file_obj_train_250)
trajectories_train_250 = [list(ele) for ele in trajectories_train_250]

file_name_test_250 = "../Human Experiments/Data/Human_trajectories_test_250.pickle"
file_obj_test_250 = open(file_name_test_250, 'rb')
trajectories_test_250 = pickle.load(file_obj_test_250)
trajectories_test_250 = [list(ele) for ele in trajectories_test_250]

file_name_train_500 = "../Human Experiments/Data/RL_trajectories_1000_500.pickle"
file_obj_train_500 = open(file_name_train_500, 'rb')
trajectories_train_500 = pickle.load(file_obj_train_500)
trajectories_train_500 = [list(ele) for ele in trajectories_train_500]

file_name_test_500 = "../Human Experiments/Data/Human_trajectories_test_500.pickle"
file_obj_test_500 = open(file_name_test_500, 'rb')
trajectories_test_500 = pickle.load(file_obj_test_500)
trajectories_test_500 = [list(ele) for ele in trajectories_test_500]

test_size = 60

In [3]:
def generate_policies(price_low_bound, price_upper_bound):
    policies = []
    for i in range(price_low_bound, price_upper_bound, 10):
        for j in range(price_low_bound, price_upper_bound, 10):
            for k in range(price_low_bound, price_upper_bound, 10):
                for l in range(5):
                    for m in range(5):
                        for n in range(5):
                            policy = (i, j, k, l, m, n)
                            policies.append(policy)
    return policies

In [4]:
policies = generate_policies(150, 260)

In [5]:
len(policies)

166375

In [6]:
def evaluate_policy(policy, data, best_buys_cost, best_buys_idx, get_final_output):
    if(len(policy)!=6):
        raise ValueError("Number of parameters in this policy is "+ str(len(policy))+" when it should be 6")
    bought = []
    bought_cost_only = []
    bought_idx_only = []
    t1 = policy[0]
    t2 = policy[1]
    t3 = policy[2]
    d1 = policy[3]
    d2 = policy[4]
    d3 = policy[5]
    for trajectory in data:
        for idx, cost in enumerate(trajectory):
            local_cat = int(idx/5)
            local_idx = int(idx%5)
            if(idx == len(trajectory) - 1):
                bought.append((cost, idx))
                bought_cost_only.append(cost)
                bought_idx_only.append(idx)
                break
            elif(local_cat == 0):
                if(cost > t1 and local_idx <= d1):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 1):
                if(cost > t2 and local_idx <= d2):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 2):
                if(cost > t3 and local_idx <= d3):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
    error = [a_i - b_i for a_i, b_i in zip(bought_cost_only, best_buys_cost)]
    correct_stops = (np.equal(bought_idx_only, best_buys_idx)).astype(int)
    correct_stops = sum(correct_stops)
    avg_error = sum(error)/len(data)
    if(get_final_output):
        return bought, avg_error, correct_stops
    return avg_error, correct_stops

In [7]:
##Cost
best_buy_250_train = [min(ele) for ele in trajectories_train_250]
best_buy_250_test = [min(ele) for ele in trajectories_test_250]
best_buy_500_train = [min(ele) for ele in trajectories_train_500]
best_buy_500_test = [min(ele) for ele in trajectories_test_500]

##Indices
best_buy_250_train_idx = [ele.index(min(ele)) for ele in trajectories_train_250]
best_buy_250_test_idx = [ele.index(min(ele)) for ele in trajectories_test_250]
best_buy_500_train_idx = [ele.index(min(ele)) for ele in trajectories_train_500]
best_buy_500_test_idx = [ele.index(min(ele)) for ele in trajectories_test_500]

In [8]:
evaluate_policy((190, 210, 230, 4, 4, 3), trajectories_train_250, best_buy_250_train, best_buy_250_train_idx, False)

(13.737, 524)

In [9]:
def readfile(filename):
    results = []
    fileObject = open(filename,'rb')  
    results = pickle.load(fileObject)
    fileObject.close()
    return results


def get_all_filenames(directoryname):
    txtfiles = []
    for file in glob.glob(directoryname):
        txtfiles.append(file)
    return txtfiles


def min_loc():
    final_minimum = readfile('../Human Experiments/Tests/Human_minimum.pickle')
    result = []
    for i, res in enumerate(final_minimum):
        temp_result = []
        for j, traj_res in enumerate(res):
            temp_result.append(traj_res[1])
        result.append(temp_result)
    return result


def get_final_results(directory):
    filenames = get_all_filenames(directory)
    final_results = [np.zeros(60),np.zeros(60)]
    final_error = [np.zeros(60),np.zeros(60)]
    final_minimum = readfile('../Human Experiments/Tests/Human_minimum.pickle')
    for filename in filenames:
        results = readfile(filename)
        for i, res in enumerate(results):
            temp_results = np.zeros(60)
            temp_errors = np.zeros(60)
            for j, traj_res in enumerate(res):
                if final_minimum[i][j] == traj_res: temp_results[j] += 1
                temp_errors[j] += traj_res[0]-final_minimum[i][j][0]
            final_results[i] = np.add(final_results[i], temp_results)
            final_error[i] = np.add(final_error[i], temp_errors)
            
    return final_results, final_error, len(filenames)

def combine_final_results(results, t250):
    final_results = np.zeros(60)
    final_error = np.zeros(60)
    final_minimum = readfile('../Human Experiments/Tests/Human_minimum.pickle')
    for j, traj_res in enumerate(results):
        if t250:
            if final_minimum[0][j] == traj_res: final_results[j] += 1
            final_error[j] += traj_res[0]-final_minimum[0][j][0]
        else:
            if final_minimum[1][j] == traj_res: final_results[j] += 1
            final_error[j] += traj_res[0]-final_minimum[1][j][0]
            
    return final_results, final_error, 1

def result_combine_LL(res, mloc, t250):
    combine = np.zeros(15)
    for j, idx in enumerate(res):
        if t250: 
            if idx: combine[mloc[0][j]] += idx
        else: 
            if idx: combine[mloc[1][j]] += idx
    return combine

def create_consolidated_LL(test_compile, test_num, human_compile, human_num, min_compile, t250=True):
    if t250: test_compile = np.divide(test_compile,min_compile[0]*test_num)
    else: test_compile = np.divide(test_compile,min_compile[1]*test_num)
    consolidated = []
    temp = []
    for j, val in enumerate(human_compile):
        if t250: temp.append((human_compile[j], min_compile[0][j]*human_num, test_compile[j]))
        else: temp.append((human_compile[j], min_compile[1][j]*human_num, test_compile[j]))
    consolidated.append(temp)
    return consolidated

def computeLogLikelihood(S, N, p):
    p = p if p > 0.0 else 0.0+1e-10
    p = p if p < 1.0 else 1.0-1e-10
    if N == 0 :result = 0
    else: result = math.log(comb(N, S)) + (S*math.log(p) + (N-S)*math.log(1.0-p))

    return result

def log_likelihood(results, t250=True):
    mloc = min_loc()
    min_compile = [np.zeros(15),np.zeros(15)]
    for i, item in enumerate(mloc):
        for idx in item:
            min_compile[i][idx] += 1
    test_res, test_err, test_num= combine_final_results(results, t250)
    human_res, human_err, human_num = get_final_results('../Human Experiments/Tests/test*')
    test_compile = result_combine_LL(test_res, mloc, t250)
    human_compile = result_combine_LL(human_res[0], mloc, t250)
    
    consolidated = create_consolidated_LL(test_compile, test_num, human_compile, human_num, min_compile)
    
    result_LL = []
    for i, traj in enumerate(consolidated):
        temp = []
        for j, val in enumerate(traj):
            temp.append(computeLogLikelihood(val[0], val[1], val[2]))
        result_LL.append(np.sum(temp)*-1)

    return result_LL

In [14]:
def policy_search(price_low_bound, price_upper_bound, data, best_buys_cost, best_buys_idx, t250 =True):
    all_errors = []
    all_correct_stops = []
    LL_human = []
    all_policies = generate_policies(price_low_bound, price_upper_bound)
    for idx, policy in enumerate(all_policies):
        bought, policy_avg_error, correct_stops = evaluate_policy(policy, data, best_buys_cost, best_buys_idx, True)
        sys.stdout.flush()
        sys.stdout.write("\rPolicies Evaluated: {}/{}".format(idx+1, len(all_policies)))
        all_errors.append(policy_avg_error)
        all_correct_stops.append(correct_stops)
        LL_human.append(log_likelihood(bought, t250))
    optimal_policy_by_ll_human = (min(LL_human), all_policies[LL_human.index(min(LL_human))])
    optimal_policy_by_avg_error = (min(all_errors), all_policies[all_errors.index(min(all_errors))])
    optimal_policy_by_opt_stop = (max(all_correct_stops), all_policies[all_correct_stops.index(max(all_correct_stops))])
    print(min(LL_human), all_policies[LL_human.index(min(LL_human))])
    return optimal_policy_by_avg_error, optimal_policy_by_opt_stop, optimal_policy_by_ll_human



# Route 1 (250)

In [17]:
optimal_policy_250 = policy_search(180, 231, trajectories_test_250, best_buy_250_test, best_buy_250_test_idx)
optimal_policy_250

Policies Evaluated: 27000/27000[120.11844661962371] (200, 230, 230, 4, 4, 3)


((13.016666666666667, (200, 230, 230, 4, 4, 3)),
 (29, (200, 220, 230, 4, 4, 3)),
 ([120.11844661962371], (200, 230, 230, 4, 4, 3)))

In [24]:
final_result_avg_250, avg_error, correct_stops = evaluate_policy((200, 210, 230, 4, 4, 3), 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(14.716666666666667, 25)

In [25]:
final_result_best_250, avg_error, correct_stops = evaluate_policy((200, 230, 230, 4, 4, 3), 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(13.016666666666667, 29)

In [26]:
final_result_opt_stop_250, avg_error, correct_stops = evaluate_policy((200, 210, 220, 4, 4, 3), 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(17.483333333333334, 24)

# Route 2 (500)

In [19]:
optimal_policy_500 = policy_search(400, 481, trajectories_test_500, best_buy_500_test, best_buy_500_test_idx, t250=False)

Policies Evaluated: 91125/91125[173.52564464800335] (430, 420, 440, 4, 4, 3)


In [21]:
optimal_policy_500

((12.8, (410, 450, 480, 4, 4, 3)),
 (36, (410, 450, 450, 4, 4, 3)),
 ([173.52564464800335], (430, 420, 440, 4, 4, 3)))

In [27]:
final_result_avg_500, avg_error, correct_stops = evaluate_policy((410, 420, 460, 4, 4, 4), 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

(19.333333333333332, 34)

In [28]:
final_result_best_500, avg_error, correct_stops = evaluate_policy((430, 420, 440, 4, 4, 3), 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

(28.166666666666668, 27)

In [29]:
final_result_opt_stop_500, avg_error, correct_stops = evaluate_policy((400, 420, 430, 4, 4, 3), 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

(30.9, 31)

# Final Result

In [30]:
rl_three_split_avg = [final_result_avg_250, final_result_avg_500]
rl_three_split_opt_stop = [final_result_opt_stop_250, final_result_opt_stop_500]
rl_three_split_best = [final_result_best_250, final_result_best_500]

In [32]:
def save_data(obj, filename):
    final_file_object = open(filename, 'wb')
    pickle.dump(obj, final_file_object)
    final_file_object.close()

In [18]:
save_data(rl_three_split_avg, "../Human Experiments/Tests/rl_three_split_avg")
save_data(rl_three_split_opt_stop, "../Human Experiments/Tests/rl_three_split_opt_stop")

In [33]:
save_data(rl_three_split_best, "../Human Experiments/Tests/rl_three_split_best")